## Imports

In [1]:
import os
import cv2
import pathlib
import itertools
import numpy as np
from matplotlib import pyplot as plt

import torch
import torchvision
import ultralytics

import preprocessing

In [ ]:
# Clone the tensorflow models repository if it doesn't already exist
if "yolov5" in pathlib.Path.cwd().parts:
  while "yolov5" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('yolov5').exists():
  !git clone --depth 1 https://github.com/ultralytics/yolov5

In [ ]:
!cp yolov5/data/VOC.yaml yolov5/VOC.yaml 

In [ ]:
device = 'cuda'
boxes = torch.tensor([[0., 1., 2., 3.]]).to(device)
scores = torch.randn(1).to(device)
iou_thresholds = 0.5

print(torchvision.ops.nms(boxes, scores, iou_thresholds))

In [2]:
ultralytics.checks()

Ultralytics YOLOv8.2.59  Python-3.9.19 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1080, 8192MiB)
Setup complete  (12 CPUs, 15.9 GB RAM, 191.0/222.3 GB disk)


## 1. Create Dataset

In [3]:
# Klassenamen
class_names = ["1", "2", "3", "4", "5", "6"]

## 2. Training  

In [9]:
ls

 Datentr�ger in Laufwerk C: ist Windows
 Volumeseriennummer: FC67-6046

 Verzeichnis von c:\Users\Alexej\Desktop\GTSRB

20.07.2024  15:11    <DIR>          .
20.07.2024  10:36    <DIR>          ..
18.07.2024  11:45               422 .gitignore
20.07.2024  15:13    <DIR>          __pycache__
07.07.2024  12:44           358.497 FasterRcnnGTSRBKeras.ipynb
07.07.2024  12:45           129.961 final_notebook.ipynb
02.07.2024  12:15    <DIR>          GTSRB
12.07.2024  12:51    <DIR>          inference
06.07.2024  16:30    <DIR>          models
07.07.2024  12:45           292.689 my_faster_rcnn.ipynb
17.07.2024  12:24    <DIR>          myModules
06.07.2024  16:28    <DIR>          pic
20.07.2024  12:30             1.363 preprocessing.py
13.05.2024  16:15             1.533 readTrafficSigns.py
12.07.2024  16:26        12.819.444 test.ipynb
20.07.2024  14:33            97.414 tf_object_detection.ipynb
12.07.2024  16:41        23.658.334 tfTest.ipynb
18.07.2024  11:45            15.324 yolo_test.i

In [11]:
os.chdir('./yolov5')

In [19]:
!cd ./ # Go into yolov5 directory

Das System kann den angegebenen Pfad nicht finden.


In [21]:
!python train.py --img 320 --batch 32 --epochs 2 --data myVOC.yaml --weights yolov5m.pt --workers 2 --project ../yoloTrain/ --name yolo5m50E320I --patience 5

train: weights=yolov5m.pt, cfg=, data=myVOC.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=2, batch_size=32, imgsz=320, rect=False, resume=True, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=../yoloTrain/, name=yolo5m50E320I, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=10, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github:  YOLOv5 is out of date by 3 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
Traceback (most recent call last):
  File "c:\Users\Alexej\Desktop\GTSRB\yolov5\train.py", line 986, in <module>
    main(opt)
  File "c:\Users\Alexej\Desktop\GTSRB\yolov5\train.py", line 64

In [31]:
!python train.py --img 320 --batch 32 --epochs 50 --data myVOC.yaml --weights ../yoloTrain/yolo5m50E320I/weights/last.pt --workers 2 --project ../yoloTrain/ --name yolo5m50E320I --patience 5

In [24]:
!python val.py --img 320 --data myVOC.yaml --weights ../yoloTrain/yolo5m50E320I/weights/best.pt --workers 2 --project ../yoloTrain/ --name yolo5m50E320I

val: data=myVOC.yaml, weights=['../yoloTrain/yolo5m50E320I/weights/best.pt'], batch_size=32, imgsz=320, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=2, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=../yoloTrain/, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5  12be499 Python-3.9.19 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1080, 8192MiB)

Fusing layers... 
Model summary: 212 layers, 20873139 parameters, 0 gradients, 47.9 GFLOPs

val: Scanning C:\Users\Alexej\Desktop\GTSRB\yoloData\Test\labels.cache... 3170 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3170/3170 [00:00<?, ?it/s]
val: Scanning C:\Users\Alexej\Desktop\GTSRB\yoloData\Test\labels.cache... 3170 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3170/3170 [00:00<?, ?it/s]

                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0%|          | 0/100 [00:00<?, ?it/s]
    

## 3. Evaluation

In [ ]:
cd .. # Go in root directory

### 3.1 Video evaluation

In [ ]:
!python detect.py --source 0  --weights ./runs/train/exp8/weights/best.pt --img 320 --conf-thres 0.6

## 3.2 Simple detection 

In [ ]:
!python detect.py --source ../yoloData/Test/images --weights ./runs/train/exp8/weights/last.pt --img 320 --save-txt --save-conf --line-thickness 1

## 4. Inference

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./runs/train/exp8/weights/best.pt', force_reload=True)
model.eval()

In [ ]:
fig, ax = plt.subplots(2,4, figsize=(20,10))
imgs = os.listdir('../yoloData/Test/images')

for idx in itertools.product(range(2),range(4)): 
    imgname = np.random.choice(imgs)
    img = cv2.imread(f'../yoloData/Test/images/{imgname}')
    results = model(img)
    ax[idx[0],idx[1]].imshow(cv2.cvtColor(np.squeeze(results.render()), cv2.COLOR_BGR2RGB))

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('./runs/detect/exp12/*.jpg'): #assuming JPG
    display(Image(filename=imageName, width=320, height=320))
    print("\n")

In [ ]:
fig, ax = plt.subplots(2,4, figsize=(20,10))
imgs = os.listdir('../yoloData/Test/images')

for idx in itertools.product(range(2),range(4)): 
    imgname = np.random.choice(imgs)
    img = cv2.imread(f'../yoloData/Test/images/{imgname}')
    results = model(img)
    ax[idx[0],idx[1]].imshow(cv2.cvtColor(np.squeeze(results.render()), cv2.COLOR_BGR2RGB))